In [1]:
# Dependencies
import pymongo
import datetime
import pandas as pd

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.se_db

# Declare the collections
players = db.players_db
nonplayers = db.nonplayers_db

In [2]:
player_path = "datacsvs/CleanPlayerData.csv"
playerdata = pd.read_csv(player_path)

playerlist = playerdata['person'].unique()
player_ids = {p:(i+1) for i,p in enumerate(playerlist)}

gamelist = playerdata['Game'].unique()
game_ids = []

for i, g in enumerate(gamelist):
    if type(g) == str:
        curr_game = {
            'str':g,
            'id':i,
            'format':g[0:2],
            'num':g[2:]
        }
        game_ids.append(curr_game)
#     else:
#         print(g)


playerdata.head()

,person,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,gtype,GameNum,Game,Broken,Role
0,Cracknut,G,L,1.0,1.0,1.0,E,N,LG,1.0,LG1,N,Tineye
1,Edgedancer,G,L,2.0,2.0,1.0,L,N,LG,1.0,LG1,N,Cop
2,Mailliw73,G,L,4.0,4.0,1.0,L,N,LG,1.0,LG1,N,Regular
3,Aether,E,W,4.0,4.0,1.0,V,N,LG,1.0,LG1,N,Smoker
4,Claincy,E,W,-,-,0.0,S,N,LG,1.0,LG1,N,Vote Cancel


In [3]:
game_ids_df = pd.DataFrame(game_ids)
# game_ids_df.head()
# gotta add other stuff / combine this into game stats thing, also Broken, start & end date, title, GM/IM, 

In [4]:
realdata = playerdata[playerdata['Alignment'] != '0']
realdata.tail()

,person,Alignment,fOutcome,Hit1,HitL,HitNum,Death,Inactive,gtype,GameNum,Game,Broken,Role
3691,Ashbringer,G,L,1.0,1.0,1.0,E,N,QF,48.0,QF48,N,-
3692,Shard of Reading,E,W,-,-,0.0,S,N,QF,48.0,QF48,N,-
3693,The Windrunner Supreme,G,L,4.0,4.0,1.0,E,N,QF,48.0,QF48,N,-
3694,Ventyl,G,L,-,-,0.0,S,N,QF,48.0,QF48,N,-
3695,Experience,G,L,1.0,1.0,1.0,L,N,QF,48.0,QF48,N,-


In [5]:
players.delete_many({})
# Verify results:
results = players.find()
for result in results:
    print(result)

In [6]:
stats_list = []


for p in realdata.iterrows():
#     print(p[1])
    row = p[1]


    try: 
        curr_id = int(game_ids_df[game_ids_df['str'] == row['Game']].index[0])
    except:
        print(row)
        
    pgstat = {
        'player_id': player_ids.get(row['person'], 0),
        'player_name': row['person'],
        'game_id': curr_id,
        'game_str': row['Game'],
        'alignment': row['Alignment'],
        'fOutcome': row['fOutcome'],
        'death': row['Death'],        
    }
    if row['HitNum'] != '-':
        pgstat['HitNum'] = row['HitNum']
        
        if row['Hit1'] != '-' :
            pgstat['Hit1'] = row['Hit1']
            pgstat['HitL'] = row['HitL']
            
    if row['Inactive'] == 'Y':
        pgstat['inactive'] = True
    else:
        pgstat['inactive'] = False
        
    if row['Role'] != '-':
        pgstat['role'] = row['Role']
    
    stats_list.append(pgstat)
    
    players.insert_one(pgstat)
    
    

In [7]:
# # Verify results:
# results = players.find()
# for result in results:
#     print(result)
from bson.json_util import dumps
cursor = players.find()
list_cur = list(cursor)
json_data = dumps(list_cur)
# Writing data to file data.json 
with open('datajsons/playerdata.json', 'w') as file: 
    file.write(json_data) 

In [8]:
nonplayer_path = "datacsvs/NonPlayerData.csv"
npdata = pd.read_csv(nonplayer_path)
npdata.tail()

,person,GM,Mod,Spec,gtype,GameNum,Game,Broken
1440,Straw,Y,N,N,LG,71.0,LG71,N
1441,Matrim's_Dice,Y,N,N,LG,71.0,LG71,N
1442,little wilson,N,Y,N,LG,71.0,LG71,N
1443,Elkanah,Y,N,N,QF,48.0,QF48,N
1444,Elbereth,N,Y,N,QF,48.0,QF48,N


In [9]:
nonplayers.delete_many({})
# Verify results:
results = nonplayers.find()
for result in results:
    print(result)

In [10]:
np_stats = []


for n in npdata.iterrows():
#     print(p[1])
    row2 = n[1]


    try: 
        curr_id2 = int(game_ids_df[game_ids_df['str'] == row2['Game']].index[0])
    except:
        print(row)
        
    npgstat = {
        'player_id': player_ids.get(row2['person'], 0),
        'player_name': row2['person'],
        'game_id': curr_id2,
        'game_str': row2['Game'],
                
    }
            
    if row2['Spec'] == 'Y':
        npgstat['GM'] = False
        npgstat['spec'] = True
        npgstat['IM'] = False
    elif row2['GM'] == 'Y':
        npgstat['GM'] = True
        npgstat['spec'] = False
        npgstat['IM'] = False
    else:
        npgstat['GM'] = False
        npgstat['spec'] = False
        npgstat['IM'] = True
        

    
    np_stats.append(npgstat)
    
    nonplayers.insert_one(npgstat)
    
    

In [11]:
# results = nonplayers.find()
# for result in results:
#     print(result)

cursor2 = nonplayers.find()
list_cur2 = list(cursor2)
json_data2 = dumps(list_cur2)
# Writing data to file data.json 
with open('datajsons/nonplayerdata.json', 'w') as file2: 
    file2.write(json_data2) 